<a href="https://colab.research.google.com/github/Tien84/BitcoinPrediction/blob/master/GridSearch_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy

In [1]:
from google.colab import drive
drive.mount('/content/driver')

Mounted at /content/driver


In [2]:
# grid search ARIMA parameters
import warnings
from math import sqrt
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


In [3]:
import pandas as pd
df=pd.read_csv('/content/driver/My Drive/DataSet/DataBitcoin.csv',header=0, index_col=0, parse_dates=True, squeeze=True)
df=df['Adj Close'].dropna()
print('Shape of data',df.shape)
df.head()
df

Shape of data (2558,)


Date
2015-11-03      403.416992
2015-11-04      411.562988
2015-11-05      386.354004
2015-11-06      374.470001
2015-11-07      386.481995
                  ...     
2022-10-30    20635.603516
2022-10-31    20495.773438
2022-11-01    20485.273438
2022-11-02    20159.503906
2022-11-03    20335.416016
Name: Adj Close, Length: 2558, dtype: float64

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(df, arima_order):
	# prepare training dataset
	df = df.astype('float32')
	train_size = int(len(df) * 0.667)
	train, test = df[0:train_size], df[train_size:]
	history = [x for x in train]
	#test.reset_index(drop=True,inplace = True)
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse

In [4]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model_mape(df, arima_order):
	# prepare training dataset
	df = df.astype('float32')
	train_size = int(len(df) * 0.667)
	train, test = df[0:train_size], df[train_size:]
	history = [x for x in train]
	#test.reset_index(drop=True,inplace = True)
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	mape = np.mean(np.abs(np.array(predictions) - np.array(test))/np.abs(test))
	print('MAPE: ' + str(mape))
	return mape

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [5]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models_mape(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mape = evaluate_arima_model_mape(dataset, order)
					if mape < best_score:
						best_score, best_cfg = mape, order
					print('ARIMA%s MAPE=%.3f' % (order,mape))
				except:
					continue
	print('Best ARIMA%s MAPE=%.3f' % (best_cfg, best_score))

In [ ]:
# load dataset
def parser(x):
	return datetime.strptime('190'+x,'%Y-%m')
# load dataset
df = pd.read_csv('/content/driver/My Drive/DataSet/DataBitcoin.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
# evaluate parameters
'''p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)'''
p_values = [0,1,2,3]
d_values = [0,1]
q_values = [0,1,2,3]

warnings.filterwarnings("ignore")
evaluate_models(df['Adj Close'].values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=28475.468
ARIMA(0, 0, 1) RMSE=14782.088
ARIMA(0, 1, 0) RMSE=1421.699
ARIMA(0, 1, 1) RMSE=1423.681
ARIMA(0, 1, 2) RMSE=1426.763
ARIMA(0, 1, 3) RMSE=1428.976
ARIMA(1, 0, 0) RMSE=1421.301
ARIMA(1, 1, 0) RMSE=1423.829
ARIMA(2, 0, 0) RMSE=1446.863
ARIMA(2, 1, 0) RMSE=1426.827
ARIMA(2, 1, 1) RMSE=1432.431
ARIMA(3, 0, 0) RMSE=3817.214
ARIMA(3, 1, 0) RMSE=1429.398
ARIMA(3, 1, 1) RMSE=1432.588
Best ARIMA(1, 0, 0) RMSE=1421.301


In [ ]:
# load dataset
def parser(x):
	return datetime.strptime('190'+x,'%Y-%m')
# load dataset
df = pd.read_csv('/content/driver/My Drive/DataSet/DataBitcoin.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
'''p_values = [0,1,2,3]
d_values = [0,1]
q_values = [0,1,2,3]'''

warnings.filterwarnings("ignore")
evaluate_models_mape(df['Adj Close'].values, p_values, d_values, q_values)